# Example: The Dynamics of American Style Options Contract Pricing
The premium of an American-style contract is _greater than or equal to_ the expected value of the discounted future payoff:

$$
\begin{equation}
\mathcal{P}_{\star}(K,T, S(0)) \geq \mathbb{E}\Bigl(\mathcal{D}^{-1}_{T,0}(\bar{r})\cdot{V_{\star}}(K,S(T))\Bigr)
\end{equation}
$$

The key difference between European and American-style option contracts is that early exercise is allowed for American-style contracts. Thus, the contract seller may demand a premium that is _greater than the European contract_ because of the possibility of early exercise. For American-style contracts, the buyer can exercise between $0\rightarrow{T}$ at any time.

## Learning objectives
In this example, we'll use the binomial lattice approximation of the share price over the interval $0\rightarrow{T}$ combined with dynamic programming to compute the premium for American-style contracts. In particular, we will focus on how the contract prices change as we move from when the contract was established toward expiration.

* Objective 1: We'll review the pricing of American `call` and `put` contracts using the Cox, Ross, and Rubinstein (CRR) model and [VLQuantitativeFinancePackage.jl](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git)
* Objective 2: We'll explore the time and volatility dependence of the pricing of American `call` and `put` contracts computed using the Cox, Ross, and Rubinstein (CRR) model and [VLQuantitativeFinancePackage.jl](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git)

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-1/Project.toml`
  [4119e0bf] ~ VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
    Updating `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-1/Manifest.toml`
  [4119e0bf] ~ VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
Precompiling project...
  ✓ VLQuantitativeFinancePackage
  1 dependency successfully precompiled in 5 seconds. 203 already precompiled.
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-1`
 

### Historical data
We downloaded the options chain for [Advanced Micro Devices (AMD)](https://en.wikipedia.org/wiki/AMD) from [barchart.com](https://www.barchart.com). For this set of values, the share price of `AMD` was $S_{\circ}$ = 117.50 USD/share, and the average implied volatility for `AMD` options with `DTE = 31 days` is `IV = 51.75%`. We load the options data using the `loadoptionsdata(...)` function and store in the `options_dataset` variable, which is an instance of `DataFrame`:

In [2]:
options_dataset = loadoptionsdata();

## Objective 1: Pricing American options contracts using the CRR model

A binomial lattice model assumes that each discrete time increment, the state of the system, e.g., the share price of equity, the spot rate, etc., can either increase by a factor $u$ with probability $p$ or decrease by a factor $d$ with probability $(1-p)$. Different models can be developed for the specific values of the tuple $(u,d,p)$. One particular model is the Cox, Ross, and Rubinstein (CRR) model:

* [Cox, J. C.; Ross, S. A.; Rubinstein, M. (1979). "Option pricing: A simplified approach". Journal of Financial Economics. 7 (3): 229. CiteSeerX 10.1.1.379.7582. doi:10.1016/0304-405X(79)90015-1](https://www.sciencedirect.com/science/article/pii/0304405X79900151?via%3Dihub)

The [CRR binomial lattice model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) was initially developed for options pricing in 1979. However, one of the critical aspects of estimating an option’s price is calculating the underlying asset’s share price. In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model, the `up` and `down` moves are symmetric:

$$ud = 1$$

where the magnitude of an `up` move $u$ is given by:

$$u = \exp(\sigma\sqrt{\Delta{T}})$$

The quantity $\sigma$ denotes a _volatility parameter_, and $\Delta{T}$ represents the time step. The probability $p$ of an `up` move in a [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) is given by:

$$p = \frac{\exp(\bar{r}\Delta{T}) - d}{u - d}$$

where $\bar{r}$ denotes a _return parameter_. In the [CRR model](https://en.wikipedia.org/wiki/Binomial_options_pricing_model) model paradigm, the return parameter $\mu$ and the volatility parameter $\sigma$ take on common values:
* The return parameter is a _risk-free_ rate of return; the _risk-free_ rate $\bar{r}$ can be approximated by the [yield on T = 10-year United States Treasury debt security](https://ycharts.com/indicators/10_year_treasury_rate). 
* The volatility parameter $\sigma$ is the [implied volatility](https://www.investopedia.com/terms/i/iv.asp); the implied volatility is the market's view of the likelihood of changes in a given security's price.

#### Implementation
Let's use the `CRR model` to simulate the premium of American-style options contracts as a function of the strike price `K`. Let's set constant values for the contracts (these values, with the exception of `IV` are the same for each strike price `K`). The time will be calendar days `Δt = 1/365`, the initial share price `Sₒ` is 117.50 USD/share, the days to expiration or `DTE` is `31` days, the annual risk-free rate is `r̄ = 0.0418` and the (average) implied volatility `IV = 51.75%`: 

In [3]:
Δt, Sₒ, r̄, IV = (1.0/365.0), 117.50, 0.0418, 0.5175;
DTE = 31.0*Δt;

Next, let's create an array of strike prices that are present in the data, we'll store this array in the `strike_price_array` variable:

In [4]:
start_strike = 40.0;
stop_strike = 200.0;
strike_price_array = range(start_strike, stop = stop_strike, step=1) |> collect;
number_of_strike_prices = length(strike_price_array);

Now that we have the strike prices, and other data we create instances of the `MyAmericanPutContractModel` and `MyAmericanCallContractModel` models which holds the contract data using the `build(...)` method. We store these instances in the `american_put_contract_model` and the `american_call_contract_model` variables:

In [5]:
american_put_contract_model = build(MyAmericanPutContractModel, (
        K = 0.0, IV = IV, DTE = DTE, sense = 1));

american_call_contract_model = build(MyAmericanCallContractModel, (
        K = 0.0, IV = IV, DTE = DTE, sense = 1));

##### Put contracts
To start the calculation for `put` contracts, we filter the `options_dataset` and collect the `put` contracts with strike prices in the `strike_price_array`, we store the `put` contract data in the `put_dataset`, which is type `DataFrame`. 

Next, we build an instance of the `MyAdjacencyBasedCRREquityPriceTree` type, which holds the parameters and data for the binomial price tree using the `build(...)` function. We store this in the `treemodel` variable. We then calculate the premium for the various strike prices in the `strike_price_array`, ranging from `K=40` to `K=200`, using a `for` loop, and record the estimated premium in the `df_CRR_put_simulation` DataFrame. For each strike price:
* We first check if the dataset has an existing `IV` value. If there is, we use that `IV` value. Otherwise, we use the average value for the implied volatility, `IV = 0.5175`. 
* Then, we update the strike price field `K` of the `put_contract_model` instance that we created above with the current strike price value. 
* Finally, we calculate the premium for this contract using the `premium(…)` function.* We first check if the dataset has an existing `IV` value. If there is, we use that `IV` value. Otherwise, we use the average value for the implied volatility, `IV = 0.5175`. 

In [ ]:
put_dataset = filter([:Type, :Strike] => (x,y)-> x == "Put" && y ∈ strike_price_array, options_dataset);
df_CRR_put_simulation = DataFrame()
for value ∈ strike_price_array
    
    # compute the IV - use average IV is not in dataset
    tmp = put_dataset[put_dataset.Strike .== value, :IV]
    IV_value = IV;
    if (isempty(tmp) == false)
        IV_value = first(tmp);
    end
    
    # build the tree -
    treemodel = build(MyAdjacencyBasedCRREquityPriceTree, μ = r̄, h = 365, 
        T = DTE, σ = first(IV_value), Sₒ = Sₒ);
    
    american_put_contract_model.K = value;   
    price_value = premium(american_put_contract_model, treemodel)

    # store the results
    results_tuple = (
        K = value,
        premium = price_value,
        Type = "Put"
    );
    push!(df_CRR_put_simulation, results_tuple);
end

(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0.49864256260778633)
(ΔT, u, d, p) = (0.00023268905986113716, 1.0079252620171597, 0.9921370538909811, 0

In [ ]:
treemodel = build(MyAdjacencyBasedCRREquityPriceTree, μ = r̄, h = 365, 
        T = DTE, σ = 0.51, Sₒ = Sₒ);

In [ ]:
american_put_contract_model.K = 110.0;   
price_value = premium(american_put_contract_model, treemodel)

##### Call contracts
To start the calculation for `call` contracts, we filter the `options_dataset` and collect the `call` contracts with strike prices in the `strike_price_array`, we store the `call` contract data in the `call_dataset`, which is type `DataFrame`. 

Next, we build an instance of the `MyAdjacencyBasedCRREquityPriceTree` type, which holds the parameters and data for the binomial price tree using the `build(...)` function. We store this in the `treemodel` variable. We then calculate the premium for the various strike prices in the `strike_price_array`, ranging from `K=40` to `K=200`, using a `for` loop, and record the estimated premium in the `df_CRR_call_simulation` DataFrame. For each strike price:
* We first check if the dataset has an existing `IV` value. If there is, we use that `IV` value. Otherwise, we use the `at the money` value for the implied volatility, `IV = 0.5175`. 
* Then, we update the strike price field `K` of the `put_contract_model` instance that we created above with the current strike price value.
* Finally, we calculate the premium for this contract using the `premium(…)` function.

In [ ]:
call_dataset = filter([:Type, :Strike] => (x,y)-> x == "Call" && y ∈ strike_price_array, options_dataset);
df_CRR_call_simulation = DataFrame()
for value ∈ strike_price_array
    
    # compute the IV - use average IV is not in dataset
    tmp = call_dataset[call_dataset.Strike .== value, :IV]
    IV_value = IV;
    if (isempty(tmp) == false)
        IV_value = first(tmp);
    end
    
    # build the tree -
    treemodel = build(MyAdjacencyBasedCRREquityPriceTree, μ = r̄, h = 365, 
        T = DTE, σ = first(IV_value), Sₒ = Sₒ);
    
    american_call_contract_model.K = value;   
    price_value = premium(american_call_contract_model, treemodel)

    # store the results
    results_tuple = (
        K = value,
        premium = price_value,
        Type = "Call"
    );
    push!(df_CRR_call_simulation, results_tuple);
end

#### Visualize
Let's plot the strike price `K` on the horizontal axis and the estimated premium on the vertical axis. 
* The CRR option price simulation captures the prices of `put` options that are `near the money`, i.e., when $S_{\circ}\simeq{K}$. Additionally, it captures the prices for constracts `in the money`, i.e., $S_{\circ} \ll {K}$ and `out of the money`, i.e., $S_{\circ}\gg{K}$:
* The CRR option price simulation predicts the prices of `call` options that are `near the money`, i.e., when $S_{\circ}\simeq{K}$. However, it underpredicts the prices of constracts `in the money`, i.e., $S_{\circ} \gg {K}$ and `out of the money`, i.e., $S_{\circ}\ll{K}$:

In [ ]:
plot(df_CRR_put_simulation[:,1],df_CRR_put_simulation[:,2], lw=3, label="Simulation Put contract")
scatter!(put_dataset[:,:Strike], put_dataset[:,:Ask],label="Put Actual DTE = 31 AMD", mc=:white, msc=:blue)
plot!(df_CRR_call_simulation[:,1],df_CRR_call_simulation[:,2], lw=3, label="Simulation Call contract", c=:red)
scatter!(call_dataset[:,:Strike], call_dataset[:,:Ask],label="Call Actual DTE = 31 AMD", mc=:white, msc=:red)

VLINE = Array{Float64,2}(undef, number_of_strike_prices, 2);
premium_low = 0.0
premium_high = maximum(put_dataset[:,:Ask]);
tmp = range(premium_low, premium_high, length = number_of_strike_prices) |> collect;
for i ∈ 1:number_of_strike_prices
    VLINE[i,1] = Sₒ
    VLINE[i,2] = tmp[i]
end
plot!(VLINE[:,1], VLINE[:,2], c=:gray69, ls=:dash, label="Sₒ = $(Sₒ) USD/share")

xlabel!("Strike price K (USD/share)", fontsize=18)
ylabel!("Option premium (USD/share)", fontsize=18)

In [ ]:
filter(:Strike=>x->x==110.0, put_dataset)

#### Aside: Visualizing the volatility smile
The volatility smile is a plot of the implied volatility levels of options with different strike prices for a specific expiration date. It takes the form of a U-shaped curve in the strike price-volatility plane, which indicates that options with varying strike prices have different implied volatility levels. 
* The smile implies that the market is anticipating higher volatility for options that are significantly out-of-the-money or in-the-money, as compared to options with strike prices near the current market price. This suggests that the market is expecting potential price movements.

##### Implementation
First, let's build an array of strike prices, called the `smile_strike_price_array`, which strike price values from `K = 60 USD` to `K = 180 USD` in increments of `5 USD`:

In [ ]:
smile_strike_price_array = range(60,180,step=5) |> collect;

Next, we'll filter the `options_dataset` to include only `call` and `put` contracts corresponding to the strike prices in the `smile_strike_price_array`. We'll store the filtered results in the `smile_dataset_call` and `smile_dataset_put` datasets:

In [ ]:
smile_dataset_call = filter([:Type, :Strike] => (x,y)-> x == "Call" && y ∈ smile_strike_price_array, options_dataset);
smile_dataset_put = filter([:Type, :Strike] => (x,y)-> x == "Put" && y ∈ smile_strike_price_array, options_dataset);

Finally, let's plot the strike price `K` versus the implied volatility `IV` for the `call` and `put` datasets, along with the `at the money (ATM)` implied volatility:

In [ ]:
plot(smile_dataset_put[:,:Strike], smile_dataset_put[:,:AVGIV], lw=3, c=:gray69, label="ATM-IV")
plot!(smile_dataset_call[:,:Strike], smile_dataset_call[:,:IV], lw=3, c=:red, label="Call")
plot!(smile_dataset_put[:,:Strike], smile_dataset_put[:,:IV], lw=3, c=:blue, label="Put")
ylabel!("Implied Volatility (IV) DTE = 31d", fontsize=18)
xlabel!("Strike prike (K)", fontsize=18)

## Objective 2: Time, Volatility and Underlying Dependence of American Option Contract Pricing
The prices for the `call` and `put` contracts calculated in `Objective 1` are _snapshots_ in time, i.e., they reflect the market conditions when the contract was purchased. However, as we move forward in time the market conditions change, the share price of the underlying asset changes, the time left on the contract decreases, and the implied volatility changes typically decreases as we move toward expiration, but an increases in the implied volatility is also possible.

In [ ]:
β₁ = 0.8; # lower perturbation bound
β₂ = 1.2; # upper perturbation bounds

Fill me in

In [ ]:
number_of_divisions = 3;
β = range(β₁, β₂, length = number_of_divisions) |> collect;
DTEₒ = DTE;
IVₒ = IV;

Fill me in

In [ ]:
design_array = Array{Float64,3}(undef, number_of_divisions, number_of_divisions, number_of_divisions);
american_put_contract_model.K = 120.0;
treemodel = build(MyAdjacencyBasedCRREquityPriceTree, μ = r̄, h = 365, 
            T = DTE, σ = IV, Sₒ = Sₒ);
baseprice = premium(american_put_contract_model, treemodel)
for i ∈ eachindex(β)
    
    DTE_value = β[i]*DTEₒ
    
    for j ∈ eachindex(β)
        
        IV_value = β[j]*IVₒ
        
        for k ∈ eachindex(β)
            
            S_value = β[k]*Sₒ
            
            treemodel = build(MyAdjacencyBasedCRREquityPriceTree, μ = r̄, h = 365, 
            T = DTE_value, σ = IV_value, Sₒ = S_value);
        
            price_value = premium(american_put_contract_model, treemodel)
            design_array[i,j,k] = log(price_value/baseprice);
        end
    end
end
design_array;

In [ ]:
design_array[:,:,3]

In [ ]:
heatmap(β, β, design_array[:,:,2], xlabel="β DTE", ylabel="β IV")
plot!(β,1.0*ones(number_of_divisions), lw=2, label="", c=:white)
plot!(1.0*ones(number_of_divisions), β, lw=2, label="", c=:white)